## Homework 5
This notebook contains the solutions for the fifth homework assignment of the Machine Learning Zoomcamp 2025 course, which can be viewed on github with the following [link](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2025/05-evaluation/homework.md). The final solutions notebook is submitted for grading at the following [link](https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw05).
### Setup
- Install `uv`: `pip install uv`
- May need to create a virtual environment in `uv`: `uv init`
- Use the following command in a terminal to launch the notebook from the `uv` virtual environment: `uv run --with jupyter jupyter lab`

In [9]:
# Add packages to the `uv` project with the --active flag so they're available in the notebook
!uv add --active "scikit-learn==1.6.1"

Resolved 6 packages in 0.56ms
Installed 5 packages in 4.21s                               
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.2
 + threadpoolctl==3.6.0


In [10]:
import pickle
import re
import sklearn

In [2]:
BOLD = "\033[1m"

### Q1. `uv` version
1. Install uv
2. Use --version to find which version of `uv` was installed

What's the version of uv?

In [3]:
uv_version_cl = !uv --version
uv_version = uv_version_cl[0].split()[1]
print(f"The `uv` version = {BOLD}{uv_version}")

The `uv` version = 0.9.5


### Q2. Use `uv` to install Scikit-Learn
Use `uv` to install Scikit-Learn version 1.6.1.

What's the first hash for Scikit-Learn you get in the lock file?
* Include the entire string starting with sha256
* Don't include quotes

In terminal, create uv project: `uv init`. Then add required version of scikit-learn: `uv add "scikit-learn==1.6.1"`

In [4]:
fname = "uv.lock"
is_package = False
is_skl = False
with open(fname, 'r') as f:
    for line in f:
        if line.startswith("[[package]]"):
            is_package = True
        if line.startswith('name = "scikit-learn"'):
            is_skl = True
        if is_package and is_skl and "hash" in line:
            hash_line = line
            break

In [5]:
match = re.search(r"hash = (.*?)(?:,|$)", hash_line)
hash_str = match.group(1).strip().replace('"', '')
print(f"The first hash value for the scikit-learn 1.6.1 package = {BOLD}{hash_str}")

The first hash value for the scikit-learn 1.6.1 package = sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e


### 3. Load model and score record
1. Write a script for loading the pipeline with pickle
2. Score this record: {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

What's the probability that this lead will convert?

In [ ]:
# Load model with the following command in terminal (only need to run once):
# !wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

In [14]:
customer = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

In [12]:
model_file = "pipeline_v1.bin"
with open(model_file, 'rb') as f_in:
    pipeline = pickle.load(f_in)

churn = pipeline.predict_proba(customer)[0, 1]

In [17]:
print(f"The probability that the customer will churn = {BOLD}{round(churn, 3)}")

The probability that the customer will churn = 0.534


### 4. Load model and score record
